<a href="https://colab.research.google.com/github/vaibram/Research/blob/main/cleaning%20and%20training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
from sklearn.preprocessing import LabelBinarizer # Label encoding, 1-hot encoding, multi-encoding
# LABEL binarizer is a 1-hot encoded MATRIX 
import cv2
import pandas as pd
import numpy as np
import random
import imutils
from imutils import paths
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from sklearn.model_selection import train_test_split
import os

In [ ]:
covid = '/content/SAMPLE'
def loadImages(path = "/content/SAMPLE"):
    temp = os.listdir(path)
    images=[]
    for i in temp:
      if(i.endswith("jpg,png")):
        images.append(i)
    print(temp)

all_images = sorted(list(paths.list_images(covid)))
all_images[:100000]


['/content/DATASET/COVID 001.jpg',
 '/content/DATASET/COVID 002.jpg',
 '/content/DATASET/COVID 003.jpg',
 '/content/DATASET/COVID 004.jpg',
 '/content/DATASET/COVID 005.jpg',
 '/content/DATASET/COVID 006.jpg',
 '/content/DATASET/COVID 007.jpg',
 '/content/DATASET/COVID 008.jpg',
 '/content/DATASET/COVID 009.jpg',
 '/content/DATASET/COVID 010.jpg',
 '/content/DATASET/COVID 011.jpg',
 '/content/DATASET/COVID 012.jpg',
 '/content/DATASET/COVID 013.jpg',
 '/content/DATASET/COVID 014.jpg',
 '/content/DATASET/COVID 015.jpg',
 '/content/DATASET/COVID 016.jpg',
 '/content/DATASET/COVID 017.jpg',
 '/content/DATASET/COVID 018.jpg',
 '/content/DATASET/COVID 019.jpg',
 '/content/DATASET/COVID 020.jpg',
 '/content/DATASET/COVID 021.jpg',
 '/content/DATASET/COVID 022.jpg',
 '/content/DATASET/COVID 023.jpg',
 '/content/DATASET/COVID 024.jpg',
 '/content/DATASET/COVID 025.jpg',
 '/content/DATASET/COVID 026.jpg',
 '/content/DATASET/COVID 027.jpg',
 '/content/DATASET/COVID 028.jpg',
 '/content/DATASET/C

In [14]:
filenames=os.listdir("/content/SAMPLE")
categories=[]
for f_name in filenames:
    category=f_name.split(' ')[0]
    
    if category=='NORMAL':
        categories.append(1)
    else:
        categories.append(0)
df=pd.DataFrame({
    'filename':filenames,
    'category':categories
})

In [15]:
df


,filename,category
0,NORMAL 002.jpg,1
1,NORMAL 023.jpg,1
2,COVID 024.jpg,0
3,NORMAL 009.jpg,1
4,COVID 016.jpg,0
...,...,...
78,NORMAL 021.jpg,1
79,COVID 047.jpg,0
80,NORMAL 027.jpg,1
81,NORMAL 024.jpg,1


In [17]:
train_df

,filename,category
67,NORMAL 004.jpg,NORMAL
56,COVID 010.jpg,COVID
40,NORMAL 025.jpg,NORMAL
9,NORMAL 020.jpg,NORMAL
61,COVID 033.jpg,COVID
...,...,...
20,COVID 019.jpg,COVID
60,NORMAL 001.jpg,NORMAL
71,COVID 017.jpg,COVID
14,NORMAL 015.jpg,NORMAL


In [18]:
from keras import backend
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential

In [36]:
Image_Width=128
Image_Height=128
Image_Size=(Image_Width,Image_Height)
Image_Channels=3

In [37]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(Image_Width,Image_Height,Image_Channels)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',
  optimizer='rmsprop',metrics=['accuracy'])

In [38]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization_32 (Batc (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 61, 61, 64)        18496     
_________________________________________________________________
batch_normalization_33 (Batc (None, 61, 61, 64)        256       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 30, 30, 64)       

In [43]:
df["category"] = df["category"].replace({0:'COVID',1:'NORMAL'})
train_df,validate_df = train_test_split(df,test_size=0.20,
  random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
total_train=train_df.shape[0]
total_validate=validate_df.shape[0]
batch_size=15

In [51]:
total_validate

17

In [69]:
train_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1
                                )
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "/content/SAMPLE",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "/content/SAMPLE", 
    x_col='filename',
    y_col='category',
    target_size=Image_Size,
    class_mode='categorical',
    batch_size=batch_size
)
test_datagen = ImageDataGenerator(rotation_range=15,
                                rescale=1./255,
                                shear_range=0.1,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                width_shift_range=0.1,
                                height_shift_range=0.1)
test_generator = train_datagen.flow_from_dataframe(train_df,
                                                 "/content/TEST-1",x_col='filename',y_col='category',
                                                 target_size=Image_Size,
                                                 class_mode='categorical',
                                                 batch_size=batch_size)

Found 66 validated image filenames belonging to 2 classes.
Found 17 validated image filenames belonging to 2 classes.
Found 66 validated image filenames belonging to 2 classes.


In [70]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_acc',patience = 2,verbose = 1,factor = 0.5,min_lr = 0.00001)
callbacks = [earlystop,learning_rate_reduction]

In [71]:
test_filenames = os.listdir("/content/SAMPLE")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [72]:
epochs=10
history = model.fit_generator(
    train_generator, 
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
4/4 [==============================] - 4s 969ms/step - loss: 0.0237 - accuracy: 1.0000 - val_loss: 13.7557 - val_accuracy: 0.6667
Epoch 2/10
4/4 [==============================] - 4s 863ms/step - loss: 0.1613 - accuracy: 0.9608 - val_loss: 10.6405 - val_accuracy: 0.7333
Epoch 3/10
4/4 [==============================] - 4s 925ms/step - loss: 0.4826 - accuracy: 0.8833 - val_loss: 11.5326 - val_accuracy: 0.6667
Epoch 4/10
4/4 [==============================] - 4s 955ms/step - loss: 0.4234 - accuracy: 0.9000 - val_loss: 11.2049 - val_accuracy: 0.6667
Epoch 5/10
4/4 [==============================] - 4s 1s/step - loss: 0.1824 - accuracy: 0.9608 - val_loss: 13.4027 - val_accuracy: 0.6667
Epoch 6/10
4/4 [==============================] - 4s 864ms/step - loss: 0.0809 - accuracy: 0.9412 - val_loss: 11.7687 - val_accuracy: 0.6000
Epoch 7/10
4/4 [==============================] - 4s 870ms/step - loss: 0.0549 - accuracy: 0.9804 - val_loss: 9.1059 - val_accuracy: 0.6667
Epoch 8/10
4/4 [=

In [73]:
test_filenames = os.listdir("/content/TEST-1")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [74]:
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [75]:
test_df['category'] = np.argmax(predict, axis=-1)
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)
test_df['category'] = test_df['category'].replace({ 'NORMAL': 1, 'COVID': 0 })

ValueError: ignored

In [ ]:
HP_epoch = 100
HP_image_dim = (96,96,3)
data = []
labels = [] 
for impath in all_images:
  img = cv2.imread(impath)
  try:
    img = cv2.imread(impath)
    resized = cv2.resize(img, (HP_image_dim[0],HP_image_dim[1]) )
  except:
        break
  imageData = img_to_array(resized)
  data.append(imageData)
  # extract label from filename (2nd last element) / \\ 
  label = impath.split(os.path.sep)[-2]
  labels.append(label)